In [1]:
from collections import Counter
import nltk
import pandas as pd
import re as regex
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from time import time
import gensim

C:\Users\Dellpc\Anaconda2\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
class TwitterData_Initialize():
    data = []
    processed_data = []
    wordlist = []

    data_model = None
    data_labels = None
    is_testing = False
    
    def initialize(self, csv_file, is_testing_set=False, from_cached=None):
        if from_cached is not None:
            self.data_model = pd.read_csv(from_cached)
            return

        self.is_testing = is_testing_set

        if not is_testing_set:
            self.data = pd.read_csv(csv_file, header=0, names=["id", "emotion", "text"])
            self.data = self.data[self.data["emotion"].isin(["positive", "negative", "neutral"])]
        else:
            self.data = pd.read_csv(csv_file, header=0, names=["id", "text"],dtype={"id":"int64","text":"str"},nrows=4000)
            not_null_text = 1 ^ pd.isnull(self.data["text"])
            not_null_id = 1 ^ pd.isnull(self.data["id"])
            self.data = self.data.loc[not_null_id & not_null_text, :]

        self.processed_data = self.data
        self.wordlist = []
        self.data_model = None
        self.data_labels = None

In [ ]:
data = TwitterData_Initialize()
data.initialize("sent_train.csv")
data.processed_data.head(5)

In [23]:
df = data.processed_data
neg = len(df[df["emotion"] == "negative"])
pos = len(df[df["emotion"] == "positive"])
neu = len(df[df["emotion"] == "neutral"])

In [24]:
print(neg,pos,neu)

956 2888 2125


In [25]:
class TwitterCleanuper:
    def iterate(self):
        for cleanup_method in [self.remove_urls,
                               self.remove_usernames,
                               self.remove_na,
                               self.remove_special_chars,
                               self.remove_numbers]:
            yield cleanup_method

    @staticmethod
    def remove_by_regex(tweets, regexp):
        tweets.loc[:, "text"].replace(regexp, "", inplace=True)
        return tweets

    def remove_urls(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"http.?://[^\s]+[\s]?"))

    def remove_na(self, tweets):
        return tweets[tweets["text"] != "Not Available"]

    def remove_special_chars(self, tweets):  # it unrolls the hashtags to normal words
        for remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
                                                                     "@", "%", "^", "*", "(", ")", "{", "}",
                                                                     "[", "]", "|", "/", "\\", ">", "<", "-",
                                                                     "!", "?", ".", "'",
                                                                     "--", "---", "#"]):
            tweets.loc[:, "text"].replace(remove, "", inplace=True)
        return tweets

    def remove_usernames(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"@[^\s]+[\s]?"))

    def remove_numbers(self, tweets):
        return TwitterCleanuper.remove_by_regex(tweets, regex.compile(r"\s?[0-9]+\.?[0-9]*"))

In [26]:
class TwitterData_Cleansing(TwitterData_Initialize):
    def __init__(self, previous):
        self.processed_data = previous.processed_data
        
    def cleanup(self, cleanuper):
        t = self.processed_data
        for cleanup_method in cleanuper.iterate():
            if not self.is_testing:
                t = cleanup_method(t)
            else:
                if cleanup_method.__name__ != "remove_na":
                    t = cleanup_method(t)

        self.processed_data = t

In [27]:
data = TwitterData_Cleansing(data)
data.cleanup(TwitterCleanuper())
data.processed_data.head(5)

C:\Users\Dellpc\Anaconda2\envs\myenv\lib\site-packages\pandas\core\generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,id,emotion,text
1,635930169241374720,neutral,IOS App Transport Security Mm need to check if...
2,635950258682523648,neutral,Mar if you have an iOS device you should downl...
3,636030803433009153,negative,my phone does not run on latest IOS which may ...
4,636100906224848896,positive,Not sure how to start your publication on iOS ...
5,636176272947744772,neutral,Two Dollar Tuesday is here with Forklift Quick...


In [28]:
class TwitterData_TokenStem(TwitterData_Cleansing):
    def __init__(self, previous):
        self.processed_data = previous.processed_data
        
    def stem(self, stemmer=nltk.PorterStemmer()):
        def stem_and_join(row):
            row["text"] = list(map(lambda str: stemmer.stem(str.lower()), row["text"]))
            return row

        self.processed_data = self.processed_data.apply(stem_and_join, axis=1)

    def tokenize(self, tokenizer=nltk.word_tokenize):
        def tokenize_row(row):
            row["text"] = tokenizer(row["text"])
            row["tokenized_text"] = [] + row["text"]
            return row

        self.processed_data = self.processed_data.apply(tokenize_row, axis=1)

In [29]:
data = TwitterData_TokenStem(data)
data.tokenize()
data.stem()
data.processed_data.head(5)

,id,emotion,text,tokenized_text
1,635930169241374720,neutral,"[io, app, transport, secur, mm, need, to, chec...","[IOS, App, Transport, Security, Mm, need, to, ..."
2,635950258682523648,neutral,"[mar, if, you, have, an, io, devic, you, shoul...","[Mar, if, you, have, an, iOS, device, you, sho..."
3,636030803433009153,negative,"[my, phone, doe, not, run, on, latest, io, whi...","[my, phone, does, not, run, on, latest, IOS, w..."
4,636100906224848896,positive,"[not, sure, how, to, start, your, public, on, ...","[Not, sure, how, to, start, your, publication,..."
5,636176272947744772,neutral,"[two, dollar, tuesday, is, here, with, forklif...","[Two, Dollar, Tuesday, is, here, with, Forklif..."


In [30]:
words = Counter()
for idx in data.processed_data.index:
    words.update(data.processed_data.loc[idx, "text"])

words.most_common(5)

[('the', 3744), ('to', 2477), ('i', 1667), ('a', 1620), ('on', 1557)]

In [31]:
stopwords=nltk.corpus.stopwords.words("english")
whitelist = ["n't", "not"]
for idx, stop_word in enumerate(stopwords):
    if stop_word not in whitelist:
        del words[stop_word]
words.most_common(5)

[('may', 1027), ('tomorrow', 764), ('day', 526), ('go', 499), ('thi', 495)]

In [32]:
class TwitterData_Wordlist(TwitterData_TokenStem):
    def __init__(self, previous):
        self.processed_data = previous.processed_data
        
    whitelist = ["n't","not"]
    wordlist = []
        
    def build_wordlist(self, min_occurrences=3, max_occurences=500, stopwords=nltk.corpus.stopwords.words("english"),
                       whitelist=None):
        self.wordlist = []
        whitelist = self.whitelist if whitelist is None else whitelist
        import os
        if os.path.isfile("wordlist.csv"):
            word_df = pd.read_csv("wordlist.csv")
            word_df = word_df[word_df["occurrences"] > min_occurrences]
            self.wordlist = list(word_df.loc[:, "word"])
            return

        words = Counter()
        for idx in self.processed_data.index:
            words.update(self.processed_data.loc[idx, "text"])

        for idx, stop_word in enumerate(stopwords):
            if stop_word not in whitelist:
                del words[stop_word]

        word_df = pd.DataFrame(data={"word": [k for k, v in words.most_common() if min_occurrences < v < max_occurences],
                                     "occurrences": [v for k, v in words.most_common() if min_occurrences < v < max_occurences]},
                               columns=["word", "occurrences"])

        word_df.to_csv("wordlist.csv", index_label="idx")
        self.wordlist = [k for k, v in words.most_common() if min_occurrences < v < max_occurences]

In [33]:
data = TwitterData_Wordlist(data)
data.build_wordlist()

In [34]:
words = pd.read_csv("wordlist.csv")
words.head()

,idx,word,occurrences
0,0,go,499
1,1,thi,495
2,2,wa,483
3,3,not,426
4,4,im,374


In [35]:
class TwitterData_BagOfWords(TwitterData_Wordlist):
    def __init__(self, previous):
        self.processed_data = previous.processed_data
        self.wordlist = previous.wordlist
    
    def build_data_model(self):
        label_column = []
        if not self.is_testing:
            label_column = ["label"]

        columns = label_column + list(
            map(lambda w: w + "_bow",self.wordlist))
        labels = []
        rows = []
        for idx in self.processed_data.index:
            current_row = []

            if not self.is_testing:
                # add label
                current_label = self.processed_data.loc[idx, "emotion"]
                labels.append(current_label)
                current_row.append(current_label)

            # add bag-of-words
            tokens = set(self.processed_data.loc[idx, "text"])
            for _, word in enumerate(self.wordlist):
                current_row.append(1 if word in tokens else 0)

            rows.append(current_row)

        self.data_model = pd.DataFrame(rows, columns=columns)
        self.data_labels = pd.Series(labels)
        return self.data_model, self.data_labels

In [36]:
data = TwitterData_BagOfWords(data)
bow, labels = data.build_data_model()
bow.head(5)

,label,go_bow,thi_bow,wa_bow,not_bow,im_bow,see_bow,time_bow,get_bow,like_bow,...,leadership_bow,snp_bow,tsiprass_bow,parliamentari_bow,alexi_bow,farag_bow,girlfriend_bow,castl_bow,crasher_bow,fiddl_bow
0,neutral,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,neutral,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,negative,0,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,positive,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,neutral,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
import random
seed = 666
random.seed(seed)

In [45]:
def test_classifier(X_train, y_train, X_test, y_test, classifier):
    print("")
    print("===============================================")
    classifier_name = str(type(classifier).__name__)
    list_of_labels = sorted(list(set(y_train)))
    model = classifier.fit(X_train, y_train)
    predictions = model.predict(X_test)

    precision = precision_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    recall = recall_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average=None, pos_label=None, labels=list_of_labels)
    print("            Negative     Neutral     Positive")
    print("F1       " + str(f1))
    print("Precision" + str(precision))
    print("Recall   " + str(recall))
    print("Accuracy " + str(accuracy))
    print("===============================================")
    print("predictions" +str(predictions))

    return precision, recall, accuracy, f1

In [46]:
from sklearn.naive_bayes import BernoulliNB
X_train, X_test, y_train, y_test = train_test_split(bow.iloc[:, 1:], bow.iloc[:, 0],
                                                    train_size=0.7, stratify=bow.iloc[:, 0],
                                                    random_state=seed)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, BernoulliNB())

C:\Users\Dellpc\Anaconda2\envs\myenv\lib\site-packages\sklearn\model_selection\_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)



            Negative     Neutral     Positive
F1       [0.38949672 0.45214221 0.71411765]
Precision[0.45408163 0.4853229  0.65978261]
Recall   [0.34099617 0.42320819 0.77820513]
Accuracy 0.5802089735709896
predictions['neutral' 'neutral' 'positive' ... 'neutral' 'neutral' 'neutral']


In [40]:
def cv(classifier, X_train, y_train):
    classifier_name = str(type(classifier).__name__)
    print("Crossvalidating " + classifier_name + "...")
    accuracy = [cross_val_score(classifier, X_train, y_train, cv=8, n_jobs=-1)]
    print("Accuracy: " + str(accuracy[0]))
    print("Average accuracy: " + str(np.array(accuracy[0]).mean()))
    return accuracy

In [41]:
nb_acc = cv(BernoulliNB(), bow.iloc[:,1:], bow.iloc[:,0])

Crossvalidating BernoulliNB...
Accuracy: [0.54639175 0.48820059 0.28023599 0.31415929 0.32743363 0.50073855
 0.47119645 0.53106509]
Average accuracy: 0.43242766840566427


In [48]:
from sklearn.ensemble import RandomForestClassifier
X_train, X_test, y_train, y_test = train_test_split(bow.iloc[:, 1:], bow.iloc[:, 0],
                                                    train_size=0.7, stratify=bow.iloc[:, 0],
                                                    random_state=seed)
precision, recall, accuracy, f1 = test_classifier(X_train, y_train, X_test, y_test, RandomForestClassifier(random_state=seed,n_estimators=403,n_jobs=-1))

C:\Users\Dellpc\Anaconda2\envs\myenv\lib\site-packages\sklearn\model_selection\_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)



            Negative     Neutral     Positive
F1       [0.25966851 0.43850267 0.69378531]
Precision[0.46534653 0.45895522 0.62020202]
Recall   [0.18007663 0.41979522 0.78717949]
Accuracy 0.5574677320221266
predictions['positive' 'neutral' 'positive' ... 'positive' 'positive' 'neutral']


In [43]:
rf_acc = cv(RandomForestClassifier(n_estimators=403,n_jobs=-1, random_state=seed),bow.iloc[:,1:], bow.iloc[:,0])

Crossvalidating RandomForestClassifier...
Accuracy: [0.51840943 0.46902655 0.31120944 0.22123894 0.33480826 0.4844904
 0.4859675  0.53698225]
Average accuracy: 0.42026659531230215
